# Test of Jinja2
> xxx

-toc: true

### 1- Data by Country

In [1]:
#hide_input
import ipywidgets as widgets
import altair as alt
from IPython.display import display
from ipywidgets import interact
import pandas as pd
from vega_datasets import data as data_world

sigma = 0.01
rho = 0.01
tab_name = 'Param_pcm1_esig_{:3.2f}_erho_{:3.2f}'.format(sigma,rho)



data = pd.read_excel("C:\\Users\\julie\\Documents\\GitHub\\test-blog\\input_data\\results_baseline.xlsx",sheet_name=tab_name)

def prepare_df(data):
    mapping_industry = pd.read_csv("C:\\Users\\julie\\Documents\\GitHub\\test-blog\\input_data\\Indcode.csv")

    old_columns = data.columns.values.tolist()
    new_columns = ['years','country','Indcode','beta', 'alpha']
    new_columns = new_columns + mapping_industry.Name.values.tolist()
    new_columns = new_columns + ['s', 'markups', 'markups_supplier', 'xshare']

    columns_dict = {old_columns[i]: new_columns[i] for i in range(len(old_columns))}

    data_renamed = data.rename(columns=columns_dict)
    data_renamed = data_renamed.merge(mapping_industry, on='Indcode', how='left')
    data_renamed = data_renamed.drop(columns = ['years'])


    data_markups = data_renamed.drop(columns=mapping_industry.Name.values.tolist()+['beta', 'alpha', 's', 'xshare'])
    data_indicators = data_renamed.drop(columns=mapping_industry.Name.values.tolist()+['markups_supplier','markups'])
    data_flows = data_renamed.drop(columns=['beta', 'alpha', 's', 'markups', 'markups_supplier', 'xshare'])
    data_flows = data_flows.rename(columns={'Name':'Buyers', 'suppliers':'Suppliers'})

    data_indicators = data_indicators.melt(['country','Name','Indcode'], var_name='indicators', value_name='value')
    data_flows = data_flows.melt(['country','Buyers', 'Indcode'], var_name='Suppliers', value_name='value')
    data_flows['Indcode_suppliers'] = [mapping_industry[mapping_industry.Name == data_flows.Suppliers[i]].Indcode for i in data_flows.index.tolist()]

    country_codes = pd.read_csv('C:\\Users\\julie\\Documents\\GitHub\\test-blog\\input_data\\countries_codes_and_coordinates.csv', 
                            )
    country_codes = country_codes.rename(columns={'Alpha-3 code':'country'})                                                                      
    data_countries = pd.DataFrame(data_renamed.country.unique(), columns=['country'])
    data_countries = data_countries.merge(country_codes, on='country', how='left')
    return data_markups, data_indicators, data_flows, data_countries
    

In [2]:
#hide_input

def prepare_graphs(data_df):
    data_markups, data_indicators, data_flows, data_countries = prepare_df(data_df)

    source = alt.topo_feature(data_world.world_110m.url, 'countries')


    alt.data_transformers.disable_max_rows()

    background = alt.Chart(source).mark_geoshape(
        fill='lightgray',
        stroke='white'
    ).properties(
        width=800,
        height=600
    ).project(type='equirectangular')

    selector = alt.selection_single(empty='all', fields=['country'],bind='legend', init={'country':'FRA'})

    # Define the 3 Charts
    chart_1 = alt.Chart(data_indicators, width=700, height=300).mark_bar().transform_filter(
        alt.FieldOneOfPredicate(field='indicators', oneOf=['s','beta'])).encode(
        x=alt.X('Name:N', sort=alt.SortField(field='Indcode', order='ascending'), axis=alt.Axis(title='Sectors', labelAngle=45)),
        y=alt.Y('value:Q',stack=None),
        opacity=alt.Opacity('indicators',scale=alt.Scale(domain=['beta','s'], range=[1.0,0.5]), legend=None),
        color=alt.Color('indicators', scale=alt.Scale(domain=['beta','s'], range=['red','blue']), legend=alt.Legend(orient="top-left"))).add_selection(selector)

    chart_2 = alt.Chart(data_markups, width=800, height=300).mark_point().encode(
        x='markups:Q',
        y='markups_supplier:Q',
        tooltip=['Name', 'country', 'markups', 'markups_supplier']).add_selection(selector)

    chart_3 = alt.Chart(data_flows,width = 500, height = 500).mark_rect().encode(
        x=alt.X('Suppliers:N', sort=alt.SortField(field='Indcode_suppliers', order='descending'), 
                axis=alt.Axis(labelAngle=45)),
        y=alt.Y('Buyers:N', sort=alt.SortField(field='Indcode', order='ascending'), 
                axis=alt.Axis(orient='right')),
        color = alt.Color('value:Q', legend=alt.Legend(orient='right')),
        tooltip=['Buyers', 'Suppliers','value']).add_selection(selector)

    # Points and text
    hover = alt.selection(type='single', on='mouseover', nearest=True,
                          fields=['Latitude (average)', 'Longitude (average)'])

    base = alt.Chart(data_countries).encode(
        longitude='Longitude (average):Q',
        latitude='Latitude (average):Q',
    )

    text = base.mark_text(dy=-5, align='right').encode(
        alt.Text('country', type='nominal'),
        opacity=alt.condition(~selector, alt.value(0), alt.value(1))
    )

    points = base.mark_point().encode(
        color=alt.condition(selector, 'country:N', alt.value('lightgray'), 
                            legend=alt.Legend(orient='top', columns=17, symbolLimit=50), title='Select Country'),
        size=alt.condition(~hover, alt.value(30), alt.value(100))
    ).add_selection(hover).add_selection(selector)

    filter_chart_1 = chart_1.add_selection(selector).transform_filter(
        selector).properties(title="Impact of each Sector by country")

    text_1 = alt.Chart().mark_text(dy=-300, size=20).encode(
        alt.Text('country', type='nominal')
    ).transform_filter(
        selector
    )

    filter_chart_2 = chart_2.add_selection(selector).transform_filter(
        selector).properties(title="Markups by country")

    filter_chart_3 = chart_3.add_selection(selector).transform_filter(
        selector).properties(title="I/O parameters by country")
    graph = ((background + points + text) & (filter_chart_1 ) & (filter_chart_2 ) & ( filter_chart_3 ))
    return graph


In [3]:
#hide_input
from ipywidgets.embed import embed_minimal_html
from IPython.display import HTML
import ipywidgets as widgets
import pandas as pd


def tab_importer(sigma,rho):
    tab_name = 'Param_pcm1_esig_{:3.2f}_erho_{:3.2f}'.format(sigma,rho)
    data = pd.read_excel("C:\\Users\\julie\\Documents\\GitHub\\test-blog\\input_data\\results_baseline.xlsx",
                         sheet_name=tab_name)
    graph = prepare_graphs(data)
    graph
    
full_widget = widgets.interact_manual(tab_importer,sigma=widgets.SelectionSlider(options=[0.01,0.7,1]), 
                        rho = widgets.SelectionSlider(options=[0.01,0.7,1]))
#embed_minimal_html('export.html', views=[full_widget], title='Widgets export')



interactive(children=(SelectionSlider(description='sigma', options=(0.01, 0.7, 1), value=0.01), SelectionSlide…